## Subsetting a Dataset for Exploration

In [1]:
import pyodbc
import pandas as pd

ModuleNotFoundError: No module named 'pyodbc'

In [2]:
conn_str = (
    r'Driver={SQL Server};'
    r'Server=oit-sql16-tst1.oit.umn.edu;'
    r'Database=CSOM_supertrailer;'
    r'Trusted_Connection=yes;'
    )

conn = pyodbc.connect(conn_str)

cursor = conn.cursor()

In [3]:
# two strategies of data subsetting:
# 1. focus on a small number of subjects, get all their data across stimuli
# 2. focus on s small number of stimuli, get all subjects' data
# Explorating 1 for now (reason: learn someone's preference / behavior from a few commercials, and apply that to predict for other/future commercials)

cursor.execute('''
SELECT *
  FROM [CSOM_supertrailer].[dbo].[harmonized_export] 
  WHERE subject_id=30
''')

data = cursor.fetchall()

In [14]:
# process the fetched data
colnames = ['study', 'subject_id', 'stimname', 'experiment_time', 'frame', 'Face_Time', 'Eye_Time', 'age', 'handedness', 'gender', 'eyewear', 'OrderStim', 'bpogx', 'bpogy', 'bpogv', 'lpd', 'lps', 'lpv', 'rpd', 'rps', 'rpv', 'LPUPILD', 'LPUPILV', 'RPUPILD', 'RPUPILV', 'anger_evidence', 'Anger_Intensity', 'Contempt_Evidence', 'Contempt_Intensity', 'Disgust_Evidence', 'Disgust_Intensity', 'Joy_Evidence', 'Joy_Intensity', 'Fear_Evidence', 'Fear_Intensity', 'Negative_Evidence', 'Negative_Intensity', 'Neutral_Evidence', 'Neutral_Intensity', 'Positive_Evidence', 'Positive_Intensity', 'Sadness_Evidence', 'Sadness_Intensity', 'Surprise_Evidence', 'Surprise_Intensity', 'PosNegAdSAM', 'ExciteCalmAdSAM', 'ImpCom', 'PosCom', 'LikeCom', 'ViewsCom', 'PreLikelyFav', 'PostLikelyFav', 'PostOften', 'LikelyFavDifference', 'Heart_Rate', 'average_heart_rate']
#print(len(colnames))
df = pd.DataFrame.from_records(data, columns = colnames)
df

,study,subject_id,stimname,experiment_time,frame,Face_Time,Eye_Time,age,handedness,gender,...,ImpCom,PosCom,LikeCom,ViewsCom,PreLikelyFav,PostLikelyFav,PostOften,LikelyFavDifference,Heart_Rate,average_heart_rate
0,facecom,30,CokeCameras2013,4.296021,13879,925.26599,937.21600,38,right,fema,...,0,0,0,0,0,0,0,0,63.157902,69.672142
1,facecom,30,CokeCameras2013,4.312988,13880,925.33197,937.23297,38,right,fema,...,0,0,0,0,0,0,0,0,63.157902,69.672142
2,facecom,30,CokeCameras2013,4.329040,13880,925.33197,937.24902,38,right,fema,...,0,0,0,0,0,0,0,0,63.157902,69.672142
3,facecom,30,CokeCameras2013,4.345032,13880,925.33197,937.26501,38,right,fema,...,0,0,0,0,0,0,0,0,63.157902,69.672142
4,facecom,30,CokeCameras2013,4.361999,13880,925.33197,937.28198,38,right,fema,...,0,0,0,0,0,0,0,0,63.157902,69.672142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34974,facecom,30,MMs2013,5.565918,19053,1270.19900,1282.12700,38,right,fema,...,0,0,0,0,0,0,0,0,67.189201,69.672142
34975,facecom,30,MMs2013,5.581909,19053,1270.19900,1282.14290,38,right,fema,...,0,0,0,0,0,0,0,0,67.189201,69.672142
34976,facecom,30,MMs2013,5.598022,19054,1270.26500,1282.15910,38,right,fema,...,0,0,0,0,0,0,0,0,67.189201,69.672142
34977,facecom,30,MMs2013,5.614990,19054,1270.26500,1282.17600,38,right,fema,...,0,0,0,0,0,0,0,0,67.189201,69.672142


In [15]:
# save for easy access
df.to_pickle("Data for Explorations/subject30")

## Model Building

### Data Preparation

In [1]:
import pandas as pd
import numpy as np
#import tensorflow as tf
#from tensorflow import keras

In [2]:
# read entire csv datasets (use carefully with memory limits)
df_raw = pd.read_csv("Data Files/harmonized_export.csv")
df_raw

# retrieve smaller pickled data
# df_raw = pd.read_pickle("Data for Explorations/subject30")
# df_raw

,study,subject_id,stimname,experiment_time,frame,Face_Time,Eye_Time,age,handedness,gender,...,ImpCom,PosCom,LikeCom,ViewsCom,PreLikelyFav,PostLikelyFav,PostOften,LikelyFavDifference,Heart_Rate,average_heart_rate
0,facecom,2,1050,-2.987991,4441,296.06601,312.18201,21,right,male,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.598999,61.886219
1,facecom,2,1050,-2.972000,4441,296.06601,312.19800,21,right,male,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.598999,61.886219
2,facecom,2,1050,-2.956009,4441,296.06601,312.21399,21,right,male,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.598999,61.886219
3,facecom,2,1050,-2.938003,4441,296.06601,312.23199,21,right,male,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.598999,61.886219
4,facecom,2,1050,-2.922989,4442,296.13300,312.24701,21,right,male,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.598999,61.886219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3587172,facecom,136,TacoBell2013,61.078003,13856,923.73199,937.60101,22,right,male,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81.632698,85.151566
3587173,facecom,136,TacoBell2013,61.093994,13856,923.73199,937.61700,22,right,male,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81.632698,85.151566
3587174,facecom,136,TacoBell2013,61.110962,13856,923.73199,937.63397,22,right,male,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81.632698,85.151566
3587175,facecom,136,TacoBell2013,61.127014,13857,923.79901,937.65002,22,right,male,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81.632698,85.151566


In [8]:
len(df_raw['subject_id'].unique())

101

In [3]:
# exploratory on outcome variables
print(df_raw['PosNegAdSAM'].describe())
print(df_raw['ExciteCalmAdSAM'].describe())

count    3.515789e+06
mean     3.610680e+00
std      2.134262e+00
min      1.000000e+00
25%      2.000000e+00
50%      3.000000e+00
75%      5.000000e+00
max      9.000000e+00
Name: PosNegAdSAM, dtype: float64
count    3.515789e+06
mean     4.897729e+00
std      2.171220e+00
min      1.000000e+00
25%      3.000000e+00
50%      5.000000e+00
75%      7.000000e+00
max      9.000000e+00
Name: ExciteCalmAdSAM, dtype: float64


In [4]:
# one-hot encode categorical variables
df = pd.get_dummies(df_raw, columns = ['handedness', 'gender', 'eyewear', 'bpogv', 'lpv', 'rpv', 'stimname'])
df['stimname'] = df_raw['stimname']

In [5]:
print(df.columns.values)

['study' 'subject_id' 'experiment_time' 'frame' 'Face_Time' 'Eye_Time'
 'age' 'OrderStim' 'bpogx' 'bpogy' 'lpd' 'lps' 'rpd' 'rps' 'LPUPILD'
 'LPUPILV' 'RPUPILD' 'RPUPILV' 'anger_evidence' 'Anger_Intensity'
 'Contempt_Evidence' 'Contempt_Intensity' 'Disgust_Evidence'
 'Disgust_Intensity' 'Joy_Evidence' 'Joy_Intensity' 'Fear_Evidence'
 'Fear_Intensity' 'Negative_Evidence' 'Negative_Intensity'
 'Neutral_Evidence' 'Neutral_Intensity' 'Positive_Evidence'
 'Positive_Intensity' 'Sadness_Evidence' 'Sadness_Intensity'
 'Surprise_Evidence' 'Surprise_Intensity' 'PosNegAdSAM' 'ExciteCalmAdSAM'
 'ImpCom' 'PosCom' 'LikeCom' 'ViewsCom' 'PreLikelyFav' 'PostLikelyFav'
 'PostOften' 'LikelyFavDifference' 'Heart_Rate' 'average_heart_rate'
 'handedness_left' 'handedness_right' 'gender_fema' 'gender_male'
 'eyewear_No' 'eyewear_Yes - contacts' 'eyewear_Yes - glasses' 'bpogv_0'
 'bpogv_1' 'lpv_0' 'lpv_1' 'rpv_0' 'rpv_1' 'stimname_1050' 'stimname_1111'
 'stimname_1201' 'stimname_1202' 'stimname_1440' 'stimnam

In [7]:
# for task 1, if we only use "evidence" measures
# feature_columns = ['age', 'handedness_right', 'gender_fema', 'eyewear_Yes - contacts', 'eyewear_Yes - glasses',
#        'bpogx', 'bpogy', 'bpogv_1', 'lpd', 'lps', 'lpv_1', 'rpd',
#        'rps', 'rpv_1', 'LPUPILD', 'LPUPILV', 'RPUPILD', 'RPUPILV',
#        'anger_evidence', 'Contempt_Evidence', 'Disgust_Evidence', 'Joy_Evidence', 'Fear_Evidence', 
#        'Negative_Evidence', 'Neutral_Evidence','Positive_Evidence', 'Sadness_Evidence', 'Surprise_Evidence']

# for task 2 specifically, we'd want to add stimname dummies as feature
feature_columns = ['age', 'handedness_right', 'gender_fema', 'eyewear_Yes - contacts', 'eyewear_Yes - glasses',
                   'bpogx', 'bpogy', 'bpogv_1', 'lpd', 'lps', 'lpv_1', 'rpd',
                   'rps', 'rpv_1', 'LPUPILD', 'LPUPILV', 'RPUPILD', 'RPUPILV',
                   'anger_evidence', 'Contempt_Evidence', 'Disgust_Evidence', 'Joy_Evidence', 'Fear_Evidence',
                   'Negative_Evidence', 'Neutral_Evidence','Positive_Evidence', 'Sadness_Evidence', 'Surprise_Evidence',
                   'stimname_Beck2013', 'stimname_BudweiserClydesdales', 'stimname_CokeCameras2013',
                   'stimname_GoDaddyPerfectMatch2', 'stimname_HyundaiSonata2013', 'stimname_KiaSorento2013',
                   'stimname_Lincoln2013', 'stimname_MMs2013', 'stimname_MercedesBenzSoul2013', 'stimname_Oreo2013',
                   'stimname_PepsiNext2013', 'stimname_PizzaHutHut2013', 'stimname_SamsungGalaxy2013',
                   'stimname_SkechersRelaxed2013', 'stimname_SpeedStick2013', 'stimname_TacoBell2013']


# Include "intensity" measures
# feature_columns = ['age', 'handedness_right', 'gender_fema', 'eyewear_Yes - contacts', 'eyewear_Yes - glasses',
#        'bpogx', 'bpogy', 'bpogv_1', 'lpd', 'lps', 'lpv_1', 'rpd',
#        'rps', 'rpv_1', 'LPUPILD', 'LPUPILV', 'RPUPILD', 'RPUPILV',
#        'anger_evidence', 'Anger_Intensity', 'Contempt_Evidence',
#        'Contempt_Intensity', 'Disgust_Evidence', 'Disgust_Intensity',
#        'Joy_Evidence', 'Joy_Intensity', 'Fear_Evidence', 'Fear_Intensity',
#        'Negative_Evidence', 'Negative_Intensity', 'Neutral_Evidence',
#        'Neutral_Intensity', 'Positive_Evidence', 'Positive_Intensity',
#        'Sadness_Evidence', 'Sadness_Intensity', 'Surprise_Evidence',
#        'Surprise_Intensity']

# include Heart_Rate as well
# feature_columns = ['age', 'handedness_right', 'gender_fema', 'eyewear_Yes - contacts', 'eyewear_Yes - glasses',
#        'bpogx', 'bpogy', 'bpogv_1', 'lpd', 'lps', 'lpv_1', 'rpd',
#        'rps', 'rpv_1', 'LPUPILD', 'LPUPILV', 'RPUPILD', 'RPUPILV',
#        'anger_evidence', 'Anger_Intensity', 'Contempt_Evidence',
#        'Contempt_Intensity', 'Disgust_Evidence', 'Disgust_Intensity',
#        'Joy_Evidence', 'Joy_Intensity', 'Fear_Evidence', 'Fear_Intensity',
#        'Negative_Evidence', 'Negative_Intensity', 'Neutral_Evidence',
#        'Neutral_Intensity', 'Positive_Evidence', 'Positive_Intensity',
#        'Sadness_Evidence', 'Sadness_Intensity', 'Surprise_Evidence',
#        'Surprise_Intensity', 'Heart_Rate']
print(len(feature_columns))

# unique simuli
print(df['stimname'].unique())
all_stim = ['Beck2013', 'BudweiserClydesdales', 'CokeCameras2013', 'GoDaddyPerfectMatch2', 'HyundaiSonata2013', 'KiaSorento2013', 'Lincoln2013', 'MMs2013', 'MercedesBenzSoul2013', 'Oreo2013', 'PepsiNext2013', 'PizzaHutHut2013', 'SamsungGalaxy2013', 'SkechersRelaxed2013', 'SpeedStick2013', 'TacoBell2013', '1050', '1111', '1201', '1202', '1440', '1441', '1460', '1640', '1710', '1721', '1750', '2010', '5594', '5628', '5700', '5890', '7018', '7020', '7057', '7205', '9295', '9301', '9326', '9590']
pic_stim = ['1050', '1111', '1201', '1202', '1440', '1441', '1460', '1640', '1710', '1721', '1750', '2010', '5594', '5628', '5700', '5890', '7018', '7020', '7057', '7205', '9295', '9301', '9326', '9590']
ad_stim = ['Beck2013', 'BudweiserClydesdales', 'CokeCameras2013', 'GoDaddyPerfectMatch2', 'HyundaiSonata2013', 'KiaSorento2013', 'Lincoln2013', 'MMs2013', 'MercedesBenzSoul2013', 'Oreo2013', 'PepsiNext2013', 'PizzaHutHut2013', 'SamsungGalaxy2013', 'SkechersRelaxed2013', 'SpeedStick2013', 'TacoBell2013']

# unique subjects
all_subjects = df['subject_id'].unique()
print(len(all_subjects))
#print(all_subjects)

44
['1050' '1201' '1440' '1441' '1710' '1750' '5628' '5890' '7018' '7057'
 '9301' '9326' 'BudweiserClydesdales' 'CokeCameras2013'
 'GoDaddyPerfectMatch2' 'HyundaiSonata2013' 'MercedesBenzSoul2013'
 'MMs2013' 'SkechersRelaxed2013' 'TacoBell2013' '1111' '1202' '1460'
 '1640' '1721' '2010' '5594' '5700' '7020' '7205' '9295' '9590' 'Beck2013'
 'KiaSorento2013' 'Lincoln2013' 'Oreo2013' 'PepsiNext2013'
 'PizzaHutHut2013' 'SamsungGalaxy2013' 'SpeedStick2013']
99


In [6]:
# subject 71 and 72 have no PosNegAdSAM or ExciteCalmAdSAM data, remove them
df = df[~df['subject_id'].isin([71,72])]
len(df['subject_id'].unique())

99

In [38]:
# Training task 1: Use all subjects, learn from subset of stimuli, predict for the rest of stimuli
# training-validation split based on stimuli
#np.random.shuffle(pic_stim)
#training_stim = pic_stim[0:6]
#validate_stim = pic_stim[6:12]
np.random.shuffle(ad_stim)
training_stim = ad_stim[0:4]
validate_stim = ad_stim[4:8]
train = df[df['stimname'].isin(training_stim)]
validate = df[df['stimname'].isin(validate_stim)]
train_features = train[feature_columns]
validate_features = validate[feature_columns]

# get training data into the tensor format (n_sample, n_timestep, n_feature)
train_x, validate_x = [], []
train_y, validate_y = [], []
max_len = 0
for s in training_stim:
    for p in train[(train['stimname'] == s)]['subject_id'].unique():
        y = train[(train['stimname'] == s) & (train['subject_id'] == p)]['PosNegAdSAM'].values[0]
        if not np.isnan(y):
            temp = train_features[(train['stimname'] == s) & (train['subject_id'] == p)].values
            #temp = temp[int(len(temp)*0.75):int(len(temp)*1),]
            train_x.append(temp)
            max_len = max(max_len, len(temp))
            train_y.append(y)
            #train_y.append(train[train['stimname'] == s]['ExciteCalmAdSAM'].values[0])
train_x = np.array(train_x, dtype = object)
train_y = np.array(train_y)
for s in validate_stim:
    for p in validate[(validate['stimname'] == s)]['subject_id'].unique():
        y = validate[(validate['stimname'] == s) & (validate['subject_id'] == p)]['PosNegAdSAM'].values[0]
        if not np.isnan(y):
            temp = validate_features[(validate['stimname'] == s) & (validate['subject_id'] == p)].values
            #temp = temp[int(len(temp)*0.75):int(len(temp)*1),]
            validate_x.append(temp)
            max_len = max(max_len, len(temp))
            validate_y.append(y)
            #validate_y.append(validate[validate['stimname'] == s]['ExciteCalmAdSAM'].values[0])
validate_x = np.array(validate_x, dtype = object)
validate_y = np.array(validate_y)
print(train_x.shape, validate_x.shape)
print(max_len)
print(train_y.shape, validate_y.shape)

(232,) (232,)
6874
(232,) (232,)


In [8]:
# Training task 2: Use all ad videos stimuli learn from subset of subjects, predict for the rest of subjects
# training-validation split based on subjects
df_video = df[df['stimname'].isin(ad_stim)]
np.random.shuffle(all_subjects)
training_sub = all_subjects[0:50]
validate_sub = all_subjects[50:101]
train = df_video[df_video['subject_id'].isin(training_sub)]
validate = df_video[df_video['subject_id'].isin(validate_sub)]

# get training data into the tensor format (n_sample, n_timestep, n_feature)
train_x, validate_x = [], []
train_y, validate_y = [], []
max_len = 0

for p in training_sub:
    for s in train[(train['subject_id'] == p)]['stimname'].unique():
        temp = train[(train['stimname'] == s) & (train['subject_id'] == p)]
        temp_x = temp[feature_columns].values
        #temp_x = temp_x[int(len(temp_x)*0.75):int(len(temp_x)*1),]
        temp_y = temp['PosNegAdSAM'].values[0]
        train_x.append(temp_x)
        max_len = max(max_len, len(temp_x))
        train_y.append(temp_y)
train_x = np.array(train_x, dtype = object)
train_y = np.array(train_y)

for p in validate_sub:
    for s in validate[(validate['subject_id'] == p)]['stimname'].unique():
        temp = validate[(validate['stimname'] == s) & (validate['subject_id'] == p)]
        temp_x = temp[feature_columns].values
        #temp_x = temp_x[int(len(temp_x)*0.75):int(len(temp_x)*1),]
        temp_y = temp['PosNegAdSAM'].values[0]
        validate_x.append(temp_x)
        max_len = max(max_len, len(temp_x))
        validate_y.append(temp_y)
validate_x = np.array(validate_x, dtype = object)
validate_y = np.array(validate_y)

print(train_x.shape, validate_x.shape)
print(max_len)
print(train_y.shape, validate_y.shape)

(400,) (392,)
6874
(400,) (392,)


In [9]:
# padding with special value
special_value = -9999.99
train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen = max_len, padding = "post", dtype = 'float64', value = special_value)
validate_x = keras.preprocessing.sequence.pad_sequences(validate_x, maxlen = max_len, padding = "post", dtype = 'float64', value = special_value)
print(train_x.shape)
print(validate_x.shape)

(400, 6874, 44)
(392, 6874, 44)


### Build a Single Model

In [14]:
# model setup
model = keras.Sequential()
model.add(keras.layers.Masking(mask_value = special_value, input_shape = (max_len, len(feature_columns))))
model.add(keras.layers.LSTM(64))
#model.add(keras.layers.GRU(64))
#model.add(keras.layers.Dense(32, activation = 'relu'))
model.add(keras.layers.Dense(1, activation = 'relu'))

In [15]:
# training configuration
# minimize MSE
model.compile(loss = keras.losses.MeanSquaredError(),
              optimizer = keras.optimizers.Adam(learning_rate=0.01))

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 6874, 44)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                27904     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 27,969
Trainable params: 27,969
Non-trainable params: 0
_________________________________________________________________


In [17]:
# training
history = model.fit(x = train_x, y = train_y, 
                    validation_data = (validate_x, validate_y),
                    epochs = 20, batch_size = 32)

Epoch 1/20
13/13 [==============================] - 52s 4s/step - loss: 3.5322 - val_loss: 3.7217
Epoch 2/20
13/13 [==============================] - 52s 4s/step - loss: 2.8686 - val_loss: 3.6556
Epoch 3/20
13/13 [==============================] - 54s 4s/step - loss: 2.7917 - val_loss: 3.6368
Epoch 4/20
13/13 [==============================] - 55s 4s/step - loss: 2.6617 - val_loss: 3.6687
Epoch 5/20
13/13 [==============================] - 55s 4s/step - loss: 2.6917 - val_loss: 3.6255
Epoch 6/20
13/13 [==============================] - 55s 4s/step - loss: 2.5507 - val_loss: 3.7409
Epoch 7/20
13/13 [==============================] - 55s 4s/step - loss: 2.6122 - val_loss: 3.6463
Epoch 8/20
13/13 [==============================] - 54s 4s/step - loss: 2.3771 - val_loss: 3.4981
Epoch 9/20
13/13 [==============================] - 55s 4s/step - loss: 2.2930 - val_loss: 3.4902
Epoch 10/20
13/13 [==============================] - 55s 4s/step - loss: 2.2007 - val_loss: 3.4329
Epoch 11/20
13/13 [

In [18]:
min(history.history['val_loss'])

3.2896206378936768

### Auto Tuning

In [72]:
import kerastuner as kt

In [83]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Masking(mask_value = special_value, input_shape = (max_len, len(feature_columns))))
    hp_units = hp.Choice('units', values = [32, 64, 128])
    model.add(keras.layers.LSTM(hp_units))
    #model.add(keras.layers.GRU(hp_units))
    model.add(keras.layers.Dense(1, activation = 'relu'))
    
    #hp_optimizer = hp.Choice('optimizer', values = ['adam', 'sgd'])
    hp_lr = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])
    
    model.compile(loss = keras.losses.MeanSquaredError(),
                  optimizer = keras.optimizers.Adam(learning_rate = hp_lr))

    return model

In [84]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_loss', 
                     max_epochs = 20,
                     factor = 2)

In [85]:
tuner.search(x = train_x, y = train_y,
             validation_data = (validate_x, validate_y),
             epochs = 20, batch_size = 16)

Epoch 1/2
24/24 [==============================] - ETA: 0s - loss: 14.20 - ETA: 21s - loss: 8.62 - ETA: 27s - loss: 6.64 - ETA: 29s - loss: 5.77 - ETA: 30s - loss: 5.57 - ETA: 29s - loss: 5.36 - ETA: 28s - loss: 5.15 - ETA: 27s - loss: 4.93 - ETA: 26s - loss: 4.63 - ETA: 24s - loss: 4.63 - ETA: 23s - loss: 4.38 - ETA: 21s - loss: 4.34 - ETA: 20s - loss: 4.26 - ETA: 18s - loss: 4.11 - ETA: 16s - loss: 4.14 - ETA: 14s - loss: 3.97 - ETA: 13s - loss: 4.05 - ETA: 11s - loss: 4.06 - ETA: 9s - loss: 3.9582 - ETA: 7s - loss: 4.006 - ETA: 5s - loss: 4.006 - ETA: 3s - loss: 4.074 - ETA: 1s - loss: 4.038 - ETA: 0s - loss: 4.032 - 64s 3s/step - loss: 4.0322 - val_loss: 3.3952
Epoch 2/2
24/24 [==============================] - ETA: 0s - loss: 2.897 - ETA: 21s - loss: 3.99 - ETA: 27s - loss: 3.55 - ETA: 29s - loss: 3.39 - ETA: 29s - loss: 3.41 - ETA: 29s - loss: 3.32 - ETA: 28s - loss: 3.22 - ETA: 27s - loss: 3.58 - ETA: 25s - loss: 3.50 - ETA: 24s - loss: 3.48 - ETA: 22s - loss: 3.43 - ETA: 21s - 

Epoch 1/2
24/24 [==============================] - ETA: 0s - loss: 11.48 - ETA: 10s - loss: 15.247 - ETA: 14s - loss: 17.808 - ETA: 14s - loss: 15.564 - ETA: 15s - loss: 14.802 - ETA: 14s - loss: 14.905 - ETA: 14s - loss: 15.327 - ETA: 13s - loss: 15.171 - ETA: 13s - loss: 14.026 - ETA: 12s - loss: 13.697 - ETA: 11s - loss: 13.045 - ETA: 10s - loss: 12.586 - ETA: 10s - loss: 12.223 - ETA: 9s - loss: 11.889 - ETA: 8s - loss: 11.79 - ETA: 7s - loss: 11.87 - ETA: 6s - loss: 12.03 - ETA: 5s - loss: 11.86 - ETA: 4s - loss: 11.46 - ETA: 3s - loss: 11.12 - ETA: 2s - loss: 10.92 - ETA: 1s - loss: 10.49 - ETA: 0s - loss: 10.19 - ETA: 0s - loss: 9.8753 - 31s 1s/step - loss: 9.8753 - val_loss: 4.2062
Epoch 2/2
24/24 [==============================] - ETA: 0s - loss: 2.759 - ETA: 10s - loss: 4.71 - ETA: 12s - loss: 4.28 - ETA: 13s - loss: 4.38 - ETA: 14s - loss: 4.31 - ETA: 13s - loss: 4.31 - ETA: 13s - loss: 4.33 - ETA: 13s - loss: 4.69 - ETA: 12s - loss: 4.37 - ETA: 11s - loss: 4.24 - ETA: 10s -

Epoch 1/2
24/24 [==============================] - ETA: 0s - loss: 11.48 - ETA: 38s - loss: 10.742 - ETA: 49s - loss: 10.489 - ETA: 52s - loss: 8.737 - ETA: 53s - loss: 7.93 - ETA: 52s - loss: 7.60 - ETA: 51s - loss: 7.24 - ETA: 49s - loss: 7.39 - ETA: 47s - loss: 7.18 - ETA: 44s - loss: 6.73 - ETA: 41s - loss: 6.27 - ETA: 38s - loss: 6.22 - ETA: 35s - loss: 6.14 - ETA: 32s - loss: 5.92 - ETA: 29s - loss: 6.00 - ETA: 26s - loss: 6.01 - ETA: 23s - loss: 5.87 - ETA: 20s - loss: 5.77 - ETA: 16s - loss: 5.63 - ETA: 13s - loss: 5.57 - ETA: 10s - loss: 5.48 - ETA: 6s - loss: 5.3775 - ETA: 3s - loss: 5.249 - ETA: 0s - loss: 5.202 - 112s 5s/step - loss: 5.2024 - val_loss: 3.2033
Epoch 2/2
24/24 [==============================] - ETA: 0s - loss: 2.576 - ETA: 39s - loss: 5.25 - ETA: 50s - loss: 4.09 - ETA: 53s - loss: 4.02 - ETA: 54s - loss: 3.68 - ETA: 53s - loss: 3.57 - ETA: 52s - loss: 3.63 - ETA: 50s - loss: 3.47 - ETA: 48s - loss: 3.84 - ETA: 45s - loss: 3.63 - ETA: 42s - loss: 3.51 - ETA: 

Epoch 1/2
24/24 [==============================] - ETA: 0s - loss: 16.53 - ETA: 28s - loss: 17.724 - ETA: 35s - loss: 17.478 - ETA: 37s - loss: 18.953 - ETA: 37s - loss: 17.780 - ETA: 36s - loss: 16.377 - ETA: 34s - loss: 16.103 - ETA: 32s - loss: 15.757 - ETA: 30s - loss: 15.170 - ETA: 28s - loss: 14.711 - ETA: 26s - loss: 13.834 - ETA: 24s - loss: 13.464 - ETA: 22s - loss: 12.675 - ETA: 20s - loss: 11.920 - ETA: 18s - loss: 11.549 - ETA: 16s - loss: 10.953 - ETA: 14s - loss: 10.807 - ETA: 12s - loss: 10.461 - ETA: 10s - loss: 10.228 - ETA: 8s - loss: 9.8804  - ETA: 6s - loss: 9.495 - ETA: 4s - loss: 9.183 - ETA: 2s - loss: 8.985 - ETA: 0s - loss: 8.729 - 69s 3s/step - loss: 8.7299 - val_loss: 3.5457
Epoch 2/2
24/24 [==============================] - ETA: 0s - loss: 2.881 - ETA: 22s - loss: 5.13 - ETA: 28s - loss: 4.54 - ETA: 30s - loss: 4.58 - ETA: 31s - loss: 4.73 - ETA: 31s - loss: 4.75 - ETA: 30s - loss: 4.35 - ETA: 29s - loss: 4.17 - ETA: 27s - loss: 4.19 - ETA: 26s - loss: 4.38 

Epoch 1/2
24/24 [==============================] - ETA: 0s - loss: 13.46 - ETA: 40s - loss: 13.491 - ETA: 51s - loss: 11.015 - ETA: 54s - loss: 11.620 - ETA: 55s - loss: 10.970 - ETA: 54s - loss: 10.782 - ETA: 53s - loss: 10.005 - ETA: 51s - loss: 9.499 - ETA: 48s - loss: 10.076 - ETA: 46s - loss: 9.517 - ETA: 43s - loss: 9.69 - ETA: 40s - loss: 9.60 - ETA: 37s - loss: 9.67 - ETA: 33s - loss: 10.097 - ETA: 30s - loss: 9.898 - ETA: 27s - loss: 9.63 - ETA: 24s - loss: 9.29 - ETA: 20s - loss: 9.31 - ETA: 17s - loss: 9.06 - ETA: 13s - loss: 9.04 - ETA: 10s - loss: 8.95 - ETA: 6s - loss: 9.0273 - ETA: 3s - loss: 8.929 - ETA: 0s - loss: 8.749 - 115s 5s/step - loss: 8.7495 - val_loss: 5.4165
Epoch 2/2
24/24 [==============================] - ETA: 0s - loss: 2.664 - ETA: 40s - loss: 3.92 - ETA: 51s - loss: 4.75 - ETA: 55s - loss: 4.86 - ETA: 56s - loss: 4.56 - ETA: 55s - loss: 4.79 - ETA: 53s - loss: 4.98 - ETA: 51s - loss: 4.83 - ETA: 49s - loss: 5.01 - ETA: 46s - loss: 5.58 - ETA: 43s - loss

Epoch 1/2
24/24 [==============================] - ETA: 0s - loss: 7.590 - ETA: 10s - loss: 8.47 - ETA: 13s - loss: 8.67 - ETA: 14s - loss: 7.94 - ETA: 14s - loss: 6.96 - ETA: 14s - loss: 6.78 - ETA: 14s - loss: 6.54 - ETA: 13s - loss: 6.17 - ETA: 13s - loss: 5.74 - ETA: 12s - loss: 5.71 - ETA: 11s - loss: 5.59 - ETA: 10s - loss: 5.58 - ETA: 9s - loss: 5.3410 - ETA: 9s - loss: 5.271 - ETA: 8s - loss: 5.081 - ETA: 7s - loss: 4.918 - ETA: 6s - loss: 4.846 - ETA: 5s - loss: 4.714 - ETA: 4s - loss: 4.676 - ETA: 3s - loss: 4.569 - ETA: 2s - loss: 4.583 - ETA: 1s - loss: 4.515 - ETA: 0s - loss: 4.483 - ETA: 0s - loss: 4.424 - 31s 1s/step - loss: 4.4247 - val_loss: 3.1637
Epoch 2/2
24/24 [==============================] - ETA: 0s - loss: 2.546 - ETA: 10s - loss: 2.45 - ETA: 13s - loss: 3.34 - ETA: 14s - loss: 3.23 - ETA: 15s - loss: 3.46 - ETA: 14s - loss: 3.28 - ETA: 14s - loss: 3.46 - ETA: 13s - loss: 3.37 - ETA: 13s - loss: 3.58 - ETA: 12s - loss: 3.44 - ETA: 11s - loss: 3.36 - ETA: 10s - 

Epoch 1/2
24/24 [==============================] - ETA: 0s - loss: 7.932 - ETA: 40s - loss: 7.23 - ETA: 52s - loss: 5.41 - ETA: 55s - loss: 5.49 - ETA: 56s - loss: 5.29 - ETA: 55s - loss: 4.77 - ETA: 54s - loss: 5.14 - ETA: 52s - loss: 4.89 - ETA: 49s - loss: 4.78 - ETA: 46s - loss: 4.62 - ETA: 44s - loss: 4.49 - ETA: 40s - loss: 4.53 - ETA: 37s - loss: 4.44 - ETA: 34s - loss: 4.40 - ETA: 31s - loss: 4.26 - ETA: 27s - loss: 4.24 - ETA: 24s - loss: 4.24 - ETA: 21s - loss: 4.21 - ETA: 17s - loss: 4.20 - ETA: 14s - loss: 4.09 - ETA: 10s - loss: 4.08 - ETA: 7s - loss: 4.0876 - ETA: 3s - loss: 3.991 - ETA: 0s - loss: 3.899 - 118s 5s/step - loss: 3.8998 - val_loss: 3.1151
Epoch 2/2
24/24 [==============================] - ETA: 0s - loss: 3.060 - ETA: 41s - loss: 2.90 - ETA: 52s - loss: 2.75 - ETA: 56s - loss: 2.71 - ETA: 57s - loss: 2.50 - ETA: 56s - loss: 2.77 - ETA: 54s - loss: 2.90 - ETA: 52s - loss: 3.11 - ETA: 50s - loss: 3.25 - ETA: 47s - loss: 3.20 - ETA: 44s - loss: 3.32 - ETA: 41s -

Epoch 1/2
24/24 [==============================] - ETA: 0s - loss: 10.35 - ETA: 10s - loss: 10.511 - ETA: 13s - loss: 10.157 - ETA: 14s - loss: 11.886 - ETA: 14s - loss: 12.546 - ETA: 14s - loss: 12.184 - ETA: 14s - loss: 11.845 - ETA: 13s - loss: 11.420 - ETA: 13s - loss: 11.507 - ETA: 12s - loss: 11.342 - ETA: 11s - loss: 11.276 - ETA: 10s - loss: 10.953 - ETA: 9s - loss: 10.666 - ETA: 9s - loss: 10.64 - ETA: 8s - loss: 10.37 - ETA: 7s - loss: 10.15 - ETA: 6s - loss: 10.08 - ETA: 5s - loss: 9.9754 - ETA: 4s - loss: 9.788 - ETA: 3s - loss: 9.679 - ETA: 2s - loss: 9.595 - ETA: 1s - loss: 9.654 - ETA: 0s - loss: 9.537 - ETA: 0s - loss: 9.491 - 31s 1s/step - loss: 9.4914 - val_loss: 7.7103
Epoch 2/2
24/24 [==============================] - ETA: 0s - loss: 7.299 - ETA: 11s - loss: 7.21 - ETA: 14s - loss: 8.21 - ETA: 14s - loss: 7.74 - ETA: 15s - loss: 7.46 - ETA: 14s - loss: 7.33 - ETA: 14s - loss: 7.78 - ETA: 13s - loss: 7.77 - ETA: 13s - loss: 7.56 - ETA: 12s - loss: 7.68 - ETA: 11s - l

Epoch 1/2
24/24 [==============================] - ETA: 0s - loss: 15.85 - ETA: 23s - loss: 14.362 - ETA: 29s - loss: 16.118 - ETA: 31s - loss: 14.204 - ETA: 31s - loss: 14.142 - ETA: 31s - loss: 12.951 - ETA: 30s - loss: 13.415 - ETA: 29s - loss: 12.725 - ETA: 27s - loss: 12.254 - ETA: 26s - loss: 12.881 - ETA: 24s - loss: 12.826 - ETA: 23s - loss: 12.868 - ETA: 21s - loss: 12.835 - ETA: 19s - loss: 12.791 - ETA: 17s - loss: 13.197 - ETA: 15s - loss: 13.074 - ETA: 13s - loss: 12.871 - ETA: 11s - loss: 12.909 - ETA: 9s - loss: 13.351 - ETA: 7s - loss: 13.21 - ETA: 5s - loss: 13.15 - ETA: 4s - loss: 13.07 - ETA: 2s - loss: 13.00 - ETA: 0s - loss: 13.01 - 67s 3s/step - loss: 13.0128 - val_loss: 9.9084
Epoch 2/2
24/24 [==============================] - ETA: 0s - loss: 15.93 - ETA: 22s - loss: 13.182 - ETA: 28s - loss: 13.843 - ETA: 30s - loss: 11.885 - ETA: 30s - loss: 11.043 - ETA: 30s - loss: 11.054 - ETA: 29s - loss: 11.344 - ETA: 28s - loss: 11.419 - ETA: 26s - loss: 10.860 - ETA: 25s

INFO:tensorflow:Oracle triggered exit


In [81]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
print(best_hps.get('units'), best_hps.get('learning_rate'))

32 0.01


In [82]:
# retrain best model
best_model = tuner.hypermodel.build(best_hps)
best_model.fit(x = train_x, y = train_y,
             validation_data = (validate_x, validate_y),
             epochs = 20, batch_size = 32)

Epoch 1/20
12/12 [==============================] - ETA: 0s - loss: 16.51 - ETA: 4s - loss: 15.56 - ETA: 6s - loss: 11.82 - ETA: 6s - loss: 9.8141 - ETA: 5s - loss: 8.523 - ETA: 5s - loss: 7.974 - ETA: 4s - loss: 7.443 - ETA: 3s - loss: 7.171 - ETA: 2s - loss: 6.906 - ETA: 1s - loss: 6.649 - ETA: 0s - loss: 6.369 - ETA: 0s - loss: 6.216 - 16s 1s/step - loss: 6.2169 - val_loss: 3.4209
Epoch 2/20
12/12 [==============================] - ETA: 0s - loss: 3.802 - ETA: 5s - loss: 3.842 - ETA: 6s - loss: 3.568 - ETA: 6s - loss: 3.605 - ETA: 5s - loss: 3.450 - ETA: 5s - loss: 3.187 - ETA: 4s - loss: 3.170 - ETA: 3s - loss: 3.374 - ETA: 2s - loss: 3.608 - ETA: 1s - loss: 3.605 - ETA: 0s - loss: 3.510 - ETA: 0s - loss: 3.486 - 15s 1s/step - loss: 3.4860 - val_loss: 3.1312
Epoch 3/20
12/12 [==============================] - ETA: 0s - loss: 4.329 - ETA: 5s - loss: 3.913 - ETA: 6s - loss: 4.170 - ETA: 6s - loss: 4.150 - ETA: 5s - loss: 3.959 - ETA: 5s - loss: 4.047 - ETA: 4s - loss: 4.009 - ETA: 3s

## SHAP Values

In [19]:
import shap

In [21]:
explainer = shap.DeepExplainer(model, train_x[:100])
shap_values = explainer.shap_values(validate_x[:50])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


AttributeError: 'TFDeep' object has no attribute 'between_tensors'

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0][0], features)